# 📘 Notebook 3: Employee Onboarding

---

## 🎯 Purpose

This notebook handles **Phase 4: Employee Master Upload** - the final phase of CRS setup.

### What You'll Create:
- Employee user accounts
- Role assignments (GRO, CSR, EMPLOYEE, etc.)
- Department & designation mappings
- Jurisdiction (boundary) assignments
- Login credentials

---

## ✅ Prerequisites

**CRITICAL:** You MUST complete these notebooks first:

- [ ] **Notebook 1 Complete** - Tenant, Departments, Designations created
- [ ] **Notebook 2 Complete** - Boundaries created
- [ ] **Role data exists** - System roles configured in MDMS
- [ ] **Employee data ready** - Names, mobile numbers, roles prepared

### Why Order Matters:
Employees need:
- **Departments** → From Notebook 1
- **Designations** → From Notebook 1
- **Boundaries** → From Notebook 2
- **Roles** → From MDMS (auto-fetched)

---



## 📋 Workflow Overview

### Step 1: Authenticate
Login to DIGIT Gateway

### Step 2: Generate Dynamic Template
System fetches live data from MDMS:
- Departments (from Notebook 1)
- Designations (from Notebook 1)
- Roles (from MDMS)
- Boundaries (from Notebook 2)

Template includes **pre-filled dropdowns** to prevent errors.

### Step 3: Fill Employee Data
Download and complete the Excel template

### Step 4: Upload & Process
System creates employees in HRMS with role & jurisdiction

### Step 5: Verify
Check creation status and handle any errors

---

## 🗂️ Employee Template Structure

### Mandatory Fields:

| Column | Description | Data Type | Example |
|--------|-------------|-----------|----------|
| User Name* | Full name | Text | John Doe |
| Mobile Number* | 10-digit unique | Numeric | 9876543210 |
| Department Name* | From Phase 3 | Dropdown | WATER DEPARTMENT |
| Designation Name* | From Phase 3 | Dropdown | engineer |
| Role Names* | Comma-separated | Multi-select | GRO,EMPLOYEE |
| Assignment From Date* | Start date | DD-MM-YYYY | 05-09-2024 |
| Date of Appointment* | Joining date | DD-MM-YYYY | 20-06-2024 |

### Optional Fields (System Defaults Applied):

| Column | Default Value |
|--------|---------------|
| Password | eGov@123 |
| Employee Status | EMPLOYED |
| Employee Type | PERMANENT |
| Gender | Male |
| Hierarchy Type | ADMIN |
| Boundary Type | City |
| Boundary Code | Tenant code |

### Auto-Generated Fields:

| Field | Logic | Example |
|-------|-------|----------|
| Employee Code | Name uppercase, spaces → underscores | JOHN_DOE |
| Tenant ID | From session | pg.citya |
| User UUID | Generated by HRMS | 1fda5623-... |
| User Type | Always EMPLOYEE | EMPLOYEE |

---

## 📝 Filling the Template

### Date Format Rules:
✅ **Correct:** `01-01-2024`, `15-06-2024`, `31-12-2024`
❌ **Wrong:** `1/1/2024`, `15-Jun-2024`, `2024-12-31`

### Mobile Number Rules:
- Must be exactly **10 digits**
- Must be **unique** (not already in system)
- No special characters or spaces

### Role Names Rules:
- Use **comma-separated** values (no spaces after comma)
- Use exact role codes from README sheet
- Common roles:
  - `GRO` - Grievance Routing Officer
  - `CSR` - Customer Service Representative
  - `EMPLOYEE` - General employee
  - `MDMS_ADMIN` - Master data admin

### Department & Designation:
- Must **exactly match** what you created in Notebook 1
- Use dropdowns (don't type manually)
- Designation must belong to selected department

---

## 🔄 Validation Layers

### Layer 1: Excel Validation
- Mandatory fields enforced
- Dropdown selection only
- 10-digit mobile format
- Date format DD-MM-YYYY
- Unique mobile within sheet

### Layer 2: Schema Validation
- Department exists in MDMS
- Designation belongs to department
- Role codes are valid
- Boundary code exists
- Dates are valid and ordered

### Layer 3: HRMS API Validation
- Mobile uniqueness (DB-level)
- Duplicate employee code
- Invalid jurisdiction
- Invalid role assignment

---

## 🚨 Common Errors

| Error Message | Cause | Solution |
|---------------|-------|----------|
| "Department not found" | Typo or not created in Notebook 1 | Verify exact spelling |
| "Mobile number already exists" | Duplicate in system | Use unique mobile |
| "Role not found" | Invalid role code | Check README sheet |
| "Invalid date format" | Wrong format | Use DD-MM-YYYY |
| "Boundary code invalid" | Not from Notebook 2 | Verify boundary exists |
| "Designation not in department" | Wrong mapping | Check department-designation link |

---

## 📊 Status Tracking

After upload, each row gets status columns:

| Column | Values | Meaning |
|--------|--------|----------|
| _STATUS | SUCCESS / EXISTS / FAILED | Processing result |
| _STATUS_CODE | 201 / 400 / 409 / 500 | HTTP response code |
| _ERROR_MESSAGE | Error text | Detailed error description |

### Color Coding:
- 🟢 **Green (SUCCESS)** - Employee created
- 🟡 **Yellow (EXISTS)** - Already exists (duplicate)
- 🔴 **Red (FAILED)** - Error occurred

---

## 🔒 Password Management

### Default Password:
If you leave Password column empty: **eGov@123**

### Custom Password Requirements:
- Minimum **8 characters**
- 1 uppercase letter
- 1 lowercase letter
- 1 number
- 1 special character

### ⚠️ Important:
After employee creation, **password cannot be updated via Excel**.
Use HRMS UI for password reset.

---

## 👥 Role & Jurisdiction Assignment

### What is Jurisdiction?
Jurisdiction maps an employee to specific boundaries (areas) where they can work.

**Example:**
- Employee: John Doe
- Role: GRO
- Jurisdiction: PG_CITYA_Z1_W1 (Ward 1)

→ John can only handle complaints from Ward 1.

### If No Boundary Specified:
Employee is assigned to **entire tenant boundary** (all areas).

---

## 🔧 Error Handling Workflow

### If Upload Has Errors:

1. **Download** processed error file
2. **Review** rows with `_STATUS = FAILED`
3. **Fix** errors based on `_ERROR_MESSAGE`
4. **Delete** status columns (_STATUS, _STATUS_CODE, _ERROR_MESSAGE)
5. **Save** corrected file
6. **Re-upload** → System skips successful rows

---

## ✅ Success Checklist

After completion, verify:

- [ ] All employees show `_STATUS = SUCCESS`
- [ ] Employee count matches your Excel
- [ ] No errors in error file
- [ ] Employees can login to CRS application
- [ ] Roles are correctly assigned
- [ ] Jurisdiction boundaries are correct

---

**Ready to onboard employees? Let's begin! 👇**

In [ ]:
# ============================================================================
# 📦 AUTOMATIC DEPENDENCY INSTALLATION
# ============================================================================
# This cell automatically installs all required packages for cross-platform compatibility
# Supports: Windows, macOS (Intel & Apple Silicon), Linux (x86 & ARM)

import sys
import subprocess
import platform
import os

def get_platform_name():
    """Get user-friendly platform name"""
    system = platform.system()
    machine = platform.machine().lower()
    
    if system == "Darwin":
        if machine in ['arm64', 'aarch64']:
            return "macOS (Apple Silicon)"
        else:
            return "macOS (Intel)"
    elif system == "Windows":
        return "Windows"
    elif system == "Linux":
        if machine in ['arm64', 'aarch64']:
            return "Linux (ARM)"
        else:
            return "Linux"
    else:
        return f"{system} ({machine})"

def install_dependencies():
    """Install all required dependencies with cross-platform support"""
    
    print("=" * 70)
    print("🔧 CRS Data Loader - Dependency Installation")
    print("=" * 70)
    print(f"📍 Platform: {get_platform_name()}")
    print(f"🐍 Python: {sys.version.split()[0]}")
    print("=" * 70)
    
    # Check if requirements.txt exists (check parent dirs)
    req_file = 'requirements.txt'
    if not os.path.exists(req_file):
        req_file = '../requirements.txt'
    
    if os.path.exists(req_file):
        print(f"✅ Found requirements.txt - Installing from file...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", req_file, "--upgrade"])
            print("\n✅ Dependencies installed from requirements.txt")
        except subprocess.CalledProcessError:
            print(f"\n⚠️  Some packages failed, trying fallback...")
            install_fallback()
    else:
        print("⚠️  requirements.txt not found - Using fallback installation...")
        install_fallback()
    
    # Enable Jupyter widgets with enhanced Linux support
    print("\n🔧 Enabling Jupyter widgets...")
    widget_success = False
    
    # Step 1: Install widget packages
    try:
        print("   Installing widget packages...")
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "ipywidgets", "jupyterlab_widgets", "--quiet"],
            timeout=30
        )
        widget_success = True
    except Exception:
        pass
    
    # Step 2: Try JupyterLab extension
    try:
        print("   Building JupyterLab extensions...")
        subprocess.check_call(
            [sys.executable, "-m", "jupyter", "lab", "build", "--minimize=False"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            timeout=90
        )
        print("   ✅ JupyterLab widgets installed")
        widget_success = True
    except Exception:
        print("   ⚠️  JupyterLab build skipped (may not be needed)")
    
    # Step 3: Try classic notebook extension
    try:
        subprocess.check_call(
            [sys.executable, "-m", "jupyter", "nbextension", "enable", "--py", "widgetsnbextension", "--sys-prefix"],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            timeout=30
        )
        print("   ✅ Classic notebook widgets enabled")
        widget_success = True
    except Exception:
        pass
    
    if widget_success:
        print("\n✅ Jupyter widgets configured")
    
    print("\n" + "=" * 70)
    print("✅ INSTALLATION COMPLETE!")
    print("=" * 70)
    
    # Show restart message if widgets were configured
    if widget_success and platform.system() == "Linux":
        print("💡 On Linux: If widgets don't appear, please:")
        print("   1. Stop this notebook (Ctrl+C in terminal)")
        print("   2. Restart: jupyter lab (or jupyter notebook)")
        print("   3. Refresh your browser (Ctrl+Shift+R)")
        print("=" * 70)
    
    print("📝 You can now run the next cells")
    print("=" * 70)

def install_fallback():
    """Fallback installation if requirements.txt is missing"""
    
    core_packages = [
        "pandas>=2.0.0",
        "openpyxl>=3.1.0",
        "xlsxwriter>=3.1.0",
        "requests>=2.31.0",
        "python-dotenv>=1.0.0",
        "notebook>=7.0.0",
        "jupyterlab>=4.0.0",
        "ipykernel>=6.25.0",
        "ipywidgets>=8.1.0",
        "jupyterlab_widgets>=3.0.0",
        "tqdm>=4.65.0",
        "plotly>=5.17.0",
        "sqlalchemy>=2.0.0",
        "typing-extensions>=4.7.0"
    ]
    
    print("\n📦 Installing core packages...")
    for package in core_packages:
        try:
            pkg_name = package.split('>=')[0]
            print(f"   Installing {pkg_name}...", end=" ")
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", package, "--quiet"],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            print("✅")
        except subprocess.CalledProcessError:
            print("⚠️")
    
    print("\n📦 Installing database adapter...")
    machine = platform.machine().lower()
    if machine not in ['aarch64', 'arm64']:
        try:
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "psycopg2-binary>=2.9.0", "--quiet"],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            print("   ✅ psycopg2-binary installed")
        except:
            print("   ⚠️  psycopg2-binary skipped (optional)")

# Run installation
try:
    install_dependencies()
except Exception as e:
    print(f"\n❌ Installation failed: {str(e)}")
    print("\n💡 Manual installation - Run in terminal:")
    print("   pip install -r requirements.txt")
    print("   jupyter lab build")


In [ ]:
# Import required packages
import pandas as pd
import json
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, FileLink
import shutil
import os
import sys
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Add parent directory to path to import unified_loader
# When running from Notebooks/ folder, parent is crs_dataLoader/
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Force reload modules to get latest changes
if 'unified_loader' in sys.modules:
    del sys.modules['unified_loader']
if 'mdms_validator' in sys.modules:
    del sys.modules['mdms_validator']
if 'excel_validator' in sys.modules:
    del sys.modules['excel_validator']

# Now import after adding to path
from unified_loader import UnifiedExcelReader, APIUploader, clean_nans

# Load environment variables from parent directory
env_path = os.path.join(parent_dir, '.env')
load_dotenv(env_path)

# Initialize Global Variables
CONFIG = {}
CONFIG_SET = False
TENANT_FILE = None
COMMON_MASTER_FILE = None
UPLOADED_TENANTS = []
SELECTED_TENANT = None
UPLOADER = None  # Global authenticated uploader instance

# Clear upload folder (in parent directory)
upload_dir = os.path.join(parent_dir, 'upload')
if os.path.exists(upload_dir):
    for filename in os.listdir(upload_dir):
        file_path = os.path.join(upload_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
    print(f"✅ Cleared all files from upload/")
else:
    os.makedirs(upload_dir)
    print(f"✅ Created upload/ directory")

print("✅ Packages loaded successfully!")
print("✅ Modules reloaded with latest changes!")
print(f"✅ Working directory: {os.getcwd()}")
print(f"✅ Parent directory (for imports): {parent_dir}")
print()
print("="*70)
print("  ⚙️ AUTHENTICATION REQUIRED")
print("="*70)
print("Please proceed to the next cell to authenticate with the gateway.")

In [ ]:
print("="*70)
print("        🔐AUTHENTICATION")
print("="*70)
print()

# Authentication widgets
base_url_input = widgets.Text(
    value="",
    description='🌐 Domain URL:',
    placeholder='e.g., https://unified-dev.digit.org',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

username_input = widgets.Text(
    value="",
    description='👤 Username:',
    placeholder='e.g., DEV_SUPER_ADMIN',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

password_input = widgets.Password(
    value="",
    description='🔑 Password:',
    placeholder='Enter password',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

user_type_input = widgets.Dropdown(
    options=['EMPLOYEE', 'CITIZEN'],
    value="EMPLOYEE",
    description='👔 User Type:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

tenant_id_input = widgets.Text(
    value="",
    description='🏛️ Tenant ID:',
    placeholder='e.g., dev, pg, pb',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

auth_button = widgets.Button(
    description='🔐 Authenticate',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

auth_output = widgets.Output()

def on_authenticate(b):
    global UPLOADER, CONFIG
    
    with auth_output:
        clear_output()
        
        # Validate inputs
        if not base_url_input.value.strip():
            print("❌ Gateway URL is required")
            return
        
        if not username_input.value.strip():
            print("❌ Username is required")
            return
        
        if not password_input.value.strip():
            print("❌ Password is required")
            return
        
        if not tenant_id_input.value.strip():
            print("❌ Tenant ID is required")
            return
        
        print("🔄 Authenticating with gateway...")
        print(f"   Domain Url: {base_url_input.value}")
        print(f"   Username: {username_input.value}")
        print(f"   User Type: {user_type_input.value}")
        print(f"   Tenant ID: {tenant_id_input.value}")
        print()
        
        try:
            # Create APIUploader with authentication
            UPLOADER = APIUploader(
                base_url=base_url_input.value.strip(),
                username=username_input.value.strip(),
                password=password_input.value.strip(),
                user_type=user_type_input.value,
                tenant_id=tenant_id_input.value.strip()
            )
            
            if UPLOADER.authenticated:
                # Save config
                CONFIG = {
                    'base_url': base_url_input.value.strip(),
                    'tenant_id': tenant_id_input.value.strip()
                }
                CONFIG_SET = True
                
                print()
                print("="*70)
                print("  ✅ AUTHENTICATION SUCCESSFUL!")
                print("="*70)
                print(f"\n👤 Logged in as: {UPLOADER.user_info.get('userName', 'Unknown')}")
                print(f"📧 Email: {UPLOADER.user_info.get('emailId', 'N/A')}")
                print(f"🏛️ Tenant: {UPLOADER.user_info.get('tenantId', 'N/A')}")
                print(f"\n🎭 Roles:")
                for role in UPLOADER.user_info.get('roles', []):
                    print(f"   • {role.get('name', 'Unknown')} ({role.get('code', 'N/A')})")
                print()
                print("="*70)
                print("  ➡️ Proceed to PHASE 1: Tenant Setup")
                print("="*70)
            else:
                print("❌ Authentication failed. Please check your credentials.")
        
        except Exception as e:
            print(f"❌ Error during authentication: {str(e)}")

auth_button.on_click(on_authenticate)

# Display UI
auth_ui = widgets.VBox([
    widgets.HTML("<h3>🔐 Step 0:  Authentication</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Authenticate with the eGov gateway to access all services</i></p>"),
    widgets.HTML("<br>"),
    base_url_input,
    widgets.HTML("<p style='font-size: 11px; color: #888;'><i>💡 Domain URL - same for all services</i></p>"),
    widgets.HTML("<br>"),
    username_input,
    password_input,
    user_type_input,
    tenant_id_input,
    widgets.HTML("<p style='font-size: 11px; color: #ff6600;'><i>⚠️ Credentials are used only for this session and not stored</i></p>"),
    widgets.HTML("<br>"),
    auth_button,
    auth_output
])

display(auth_ui)

In [ ]:
# Dependency Check: Verify prerequisites exist
print("="*70)
print("        ✅ CHECKING PREREQUISITES")
print("="*70)
print()

# Check if authentication was successful
if not UPLOADER or not UPLOADER.authenticated:
    print("❌ ERROR: Authentication failed!")
    print("💡 Please check your credentials and try again.")
    raise SystemExit("Authentication required")

print("✅ Authentication successful")
print("\n📋 This notebook requires:")
print("   1. Tenant must exist (from Notebook 1)")
print("   2. Departments & Designations must exist (from Notebook 1)")
print("   3. Boundaries must exist (from Notebook 2)")
print("\n💡 TIP: If you haven't run Notebooks 1 and 2, please run them first.")
print("\n✅ Prerequisites acknowledged - ready to proceed!")

# Optional: Add actual validation if needed
# Example validation code (uncomment and customize):
# try:
#     # Check departments
#     depts = UPLOADER.fetch_departments('pg.citya')  # Replace with your tenant
#     if not depts:
#         print("\n❌ ERROR: No departments found!")
#         print("💡 Please run Notebook 1 (TenantAndCommonMaster) first")
#         raise SystemExit("Missing dependency: Departments")
#     print(f"\n✅ Found {len(depts)} department(s)")
#     
#     # Check boundaries
#     boundaries = UPLOADER.fetch_boundaries('pg.citya', 'ADMIN')  # Replace with your tenant and hierarchy
#     if not boundaries:
#         print("\n❌ ERROR: No boundaries found!")
#         print("💡 Please run Notebook 2 (BoundarySetup) first")
#         raise SystemExit("Missing dependency: Boundaries")
#     print(f"✅ Found {len(boundaries)} boundary(ies)")
# except Exception as e:
#     print(f"\n⚠️  Warning: Could not verify prerequisites: {e}")
#     print("Proceeding anyway...")


---

# 🟦 PHASE 4: EMPLOYEE BULK CREATION

In Phase 4, you will bulk create employees for your tenant.

## 📋 Overview:
1. **Generate Dynamic Template** - Creates Excel with dropdowns pre-filled from MDMS
2. **Fill Employee Data** - Use the template to add employee details
3. **Upload Excel** - Upload the filled template
4. **Auto Role Validation** - System checks & creates missing roles in MDMS
5. **Bulk Create** - Creates all employees via HRMS API
6. **Status Tracking** - Excel updated with color-coded status (🟢 SUCCESS, 🟡 EXISTS, 🔴 FAILED)

## 🎯 Key Features:
- ✅ **Name-based Excel** - Use department/designation/role NAMES, system converts to codes
- ✅ **Date pickers** - Excel date inputs instead of timestamps
- ✅ **README - Roles sheet** - Copy-paste ready role names
- ✅ **Auto role creation** - Missing roles automatically created in MDMS
- ✅ **Error handling** - Status appended to each row with details

## 📖 Template Sheets:
1. **Instructions** - Complete usage guide
2. **README - Roles** - All available roles to copy from
3. **Employee Master** - Main data entry sheet
4. **Hidden reference sheets** - Departments, Designations, Roles, Boundaries


---

## 🎨 Step 4.1: Generate Dynamic Employee Template

**Instructions:**
1. Enter the **Target Tenant ID** (e.g., `pg.citya`) in the text box
2. Click **🎨 Generate Dynamic Template** button
3. Wait for template generation (may take 10-20 seconds)

**What happens:**
- System fetches **Departments** from MDMS → Creates dropdown
- System fetches **Designations** from MDMS → Creates dropdown
- System fetches **Roles** from MDMS → Creates README sheet
- System fetches **Boundaries** from Boundary Service → Creates dropdown
- Creates **Employee Master** sheet with data validation
- Creates **Instructions** sheet with usage guide
- Creates **README - Roles** sheet with all available roles

**Generated File Location:**
```
templates/Employee_Master_Dynamic_{tenant}.xlsx
```

**Template Contains:**
- ✅ All dropdowns pre-filled from MDMS
- ✅ Excel date pickers for date fields
- ✅ README - Roles sheet with copy-paste ready role names
- ✅ Instructions sheet with complete guidance
- ✅ Sample row with default values

💡 **Tip**: This template is customized for your tenant with real data from MDMS!


In [ ]:
print("="*70)
print("        🎨  GENERATE DYNAMIC EMPLOYEE TEMPLATE FROM MDMS")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 5).")
    raise SystemExit("Authentication required")

print()

# Tenant input for template generation
template_tenant_input = widgets.Text(
    value=CONFIG.get('tenant_id') or 'pg',
    placeholder='e.g., pg.citya, pb.amritsar',
    description='🏛️ Tenant ID:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# Generate button
generate_employee_template_btn = widgets.Button(
    description='🎨 Generate Dynamic Template',
    button_style='primary',
    layout=widgets.Layout(width='95%', height='40px')
)

template_gen_output = widgets.Output()

GENERATED_EMP_TEMPLATE_PATH = None

def on_generate_emp_template(b):
    global GENERATED_EMP_TEMPLATE_PATH
    
    with template_gen_output:
        clear_output()
        
        if not template_tenant_input.value.strip():
            print("❌ Please enter a tenant ID")
            return
        
        tenant = template_tenant_input.value.strip().lower()
        
        print(f"🔄 Generating dynamic template for tenant: {tenant}")
        print("   Fetching data from MDMS...\n")
        
        uploader = UPLOADER
        
        try:
            template_path = uploader.generate_employee_template(tenant)
            GENERATED_EMP_TEMPLATE_PATH = template_path
            
            download_link_html = f"""
            <div style='background:#d4edda; padding:15px; border-radius:8px; margin:10px 0;'>
                <h3 style='margin-top:0; color:#155724;'>✅ Dynamic Template Generated!</h3>
                <p><b>File:</b> {template_path}</p>
                <a href='{template_path}' download='Employee_Master_Dynamic.xlsx' 
                   style='display:inline-block; padding:12px 24px; background:#28a745; color:white; 
                          text-decoration:none; border-radius:6px; font-weight:bold; margin:10px 0;'>
                    📥 Download Template
                </a>
                <p style='margin-top:15px;'><b>✨ Features:</b></p>
                <ul style='margin:5px 0;'>
                    <li>✅ Departments fetched from MDMS</li>
                    <li>✅ Designations fetched from MDMS</li>
                    <li>✅ Roles fetched from User Service</li>
                    <li>✅ Boundaries fetched from Boundary Service</li>
                    <li>✅ 8 pre-filled dropdowns</li>
                    <li>✅ Employee code auto-generated</li>
                    <li>✅ Sample row included</li>
                </ul>
            </div>
            """
            display(HTML(download_link_html))
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")

generate_employee_template_btn.on_click(on_generate_emp_template)

if 'UPLOADED_TENANTS' in locals() and UPLOADED_TENANTS:
    emp_ref = widgets.HTML(
        f"<p style='background:#f0f0f0; padding:10px; border-radius:5px;'>"
        f"<b>📋 Tenants from Phase 1:</b> {', '.join(UPLOADED_TENANTS)}</p>"
    )
else:
    emp_ref = widgets.HTML("")

display(widgets.VBox([
    widgets.HTML("<h3>🎨 Step 4.0: Generate Dynamic Template</h3>"),
    widgets.HTML("<div style='background:#fff3cd;padding:12px;border-radius:5px;'>"
                 "<b>💡 Recommended:</b> Generate template with live MDMS data - dropdowns auto-filled!</div>"),
    emp_ref,
    widgets.HTML("<br>"),
    template_tenant_input,
    widgets.HTML("<br>"),
    generate_employee_template_btn,
    template_gen_output
]))

---

### 📝 NEXT STEPS: Fill the Template

**After generating the template:**

1. **Open the Excel file** from the `templates/` folder
2. **Read the 'Instructions' sheet** for detailed guidance
3. **Check 'README - Roles' sheet** for available role names
4. **Fill the 'Employee Master' sheet** with employee data:
   - Use NAMES for departments, designations, and roles (NOT codes)
   - Use Excel date picker for dates
   - For multiple roles: copy from README and join with commas (e.g., `Employee,CRS Viewer,CRS Admin`)
5. **Delete the sample row** before uploading
6. **Save the file** and proceed to Step 4.1 below

💡 **TIP**: The 'README - Roles' sheet has all available roles with descriptions - just copy the role names you need!

In [ ]:
print("="*70)
print("        👥  PHASE 4: UPLOAD EMPLOYEE MASTER")
print("="*70)
print()

# Global variable for employee file
EMPLOYEE_MASTER_FILE = None
EMPLOYEE_TENANT = None

# Tenant input widget
employee_tenant_input = widgets.Text(
    value=CONFIG.get('tenant_id') or 'pg',
    placeholder='e.g., pg.citya, pb.amritsar',
    description='🏛️ Tenant ID:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='95%')
)

# File upload widget
employee_file_upload = widgets.FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Select File:',
    layout=widgets.Layout(width='70%')
)

# Upload button
employee_upload_btn = widgets.Button(
    description='⬆️ Set Tenant & Upload Employee Master',
    button_style='success',
    layout=widgets.Layout(width='95%', height='40px')
)

employee_upload_output = widgets.Output()

def on_employee_upload(b):
    global EMPLOYEE_MASTER_FILE, EMPLOYEE_TENANT
    
    with employee_upload_output:
        clear_output()
        
        # Validate tenant input
        if not employee_tenant_input.value.strip():
            print("❌ Please enter a tenant ID")
            return
        
        # Validate file selection
        if not employee_file_upload.value:
            print("❌ Please select Employee Master Excel file")
            return
        
        # Set tenant
        EMPLOYEE_TENANT = employee_tenant_input.value.strip().lower()
        
        # Save file
        os.makedirs('upload', exist_ok=True)
        uploaded_file = employee_file_upload.value[0]
        content = uploaded_file['content']
        filename = 'Employee_Master.xlsx'
        
        upload_path = os.path.join('upload', filename)
        with open(upload_path, 'wb') as f:
            f.write(content)
        
        EMPLOYEE_MASTER_FILE = upload_path
        
        print("="*70)
        print("  ✅ FILE UPLOADED SUCCESSFULLY")
        print("="*70)
        print(f"\n🎯 Target Tenant: {EMPLOYEE_TENANT}")
        print(f"\n📄 File: {upload_path}")
        print("\n➡️  Proceed to bulk create employees")
        print("="*70)

employee_upload_btn.on_click(on_employee_upload)

# Show uploaded tenants as reference (if available)
if UPLOADED_TENANTS:
    reference_info = widgets.HTML(
        f"<p style='color: #666; background-color: #f0f0f0; padding: 10px; border-radius: 5px;'>"
        f"<b>📋 Reference - Uploaded Tenants from Phase 1:</b><br>"
        f"{', '.join(UPLOADED_TENANTS)}</p>"
    )
else:
    reference_info = widgets.HTML("")

# Display
employee_upload_box = widgets.VBox([
    widgets.HTML("<h3>👥 Step 4.1: Upload Employee Master</h3>"),
    widgets.HTML("<p style='color: #666;'><i>Upload Employee Master Excel to bulk create employees</i></p>"),
    reference_info,
    widgets.HTML("<br>"),
    employee_tenant_input,
    widgets.HTML("<p style='font-size: 12px; color: #888;'><i>💡 Employees will be created for this tenant</i></p>"),
    widgets.HTML("<br>"),
    widgets.HTML("<p style='color: #666;'><b>Select Employee Master Excel File:</b></p>"),
    employee_file_upload,
    widgets.HTML("<br>"),
    employee_upload_btn,
    employee_upload_output
])

display(employee_upload_box)

---

### ⚡ Step 4.2: Bulk Create Employees

**What happens when you run the cell below:**

#### 🔐 Phase 1: Role Validation (Automatic)
- System checks if all required roles exist in MDMS
- If roles are missing, they are **automatically created** from default-data-handler configuration
- Ensures HRMS validation won't fail due to missing roles

#### 👥 Phase 2: Employee Creation
- Converts all NAMES to CODES internally (departments, designations, roles)
- Converts Excel dates to timestamps
- Auto-generates employee codes from user names
- Creates jurisdiction from boundary + roles
- Calls HRMS API for each employee

#### 📊 Phase 3: Status Tracking
- Updates your Excel file with 3 new columns:
  - `_STATUS` (SUCCESS/EXISTS/FAILED)
  - `_STATUS_CODE` (HTTP code)
  - `_ERROR_MESSAGE` (error details)
- Color codes rows:
  - 🟢 **GREEN** = Successfully created
  - 🟡 **YELLOW** = Already exists (duplicate)
  - 🔴 **RED** = Failed (see error message)

#### 📈 Final Summary
- Shows count of created/exists/failed employees
- Opens updated Excel file for review

**Ready? Run the cell below to start bulk creation! ⬇️**


In [ ]:
print("="*70)
print("[PHASE 4] BULK CREATING EMPLOYEES")
print("="*70)

if not UPLOADER or not UPLOADER.authenticated:
    print("❌ Please authenticate first! Run the Authentication cell (Cell 3).")
    raise SystemExit("Authentication required")


if not EMPLOYEE_MASTER_FILE:
    print("❌ Please upload Employee Master Excel first!")
elif not EMPLOYEE_TENANT:
    print("❌ Please set target tenant first!")
else:
    # Initialize reader and uploader
    reader = UnifiedExcelReader(EMPLOYEE_MASTER_FILE)
    uploader = UPLOADER
    
    # Read employees using bulk method - PASS uploader parameter
    employees = reader.read_employees_bulk(EMPLOYEE_TENANT, uploader=uploader)
    
    print(f"\n[INFO] Loaded {len(employees)} employee(s) from Excel")
    for emp in employees[:5]:
        dept = emp['assignments'][0]['department']
        desig = emp['assignments'][0]['designation']
        roles = [r['code'] for r in emp['user']['roles']]
        print(f"   - {emp['code']}: {emp['user']['name']} [{dept}/{desig}] Roles: {', '.join(roles)}")
    if len(employees) > 5:
        print(f"   ... and {len(employees) - 5} more")
    
    # Create employees
    result_employees = uploader.create_employees(
        employee_list=clean_nans(employees),
        tenant=EMPLOYEE_TENANT,
        sheet_name='Employee Master',
        excel_file=EMPLOYEE_MASTER_FILE
    )
    
    # Summary
    if result_employees['failed'] == 0:
        print("\n✅ [SUCCESS] Employees created successfully!")
        print("\n🎉 PHASE 4 COMPLETED!")
    else:
        print("\n⚠️  [WARNING] Some employees failed. Check errors/ folder or status columns in Excel.")

In [ ]:
from IPython.display import display, HTML, FileLink
from datetime import datetime
import os

# Collect results
summary_data = []

if 'result_tenants' in locals():
    summary_data.append({
        'module': 'Tenants',
        'created': result_tenants.get('created', 0),
        'exists': result_tenants.get('exists', 0),
        'failed': result_tenants.get('failed', 0),
        'excel_file': TENANT_FILE if 'TENANT_FILE' in locals() else None,
        'sheet': 'Tenant Info'
    })

if 'result_dept' in locals():
    summary_data.append({
        'module': 'Departments',
        'created': result_dept.get('created', 0),
        'exists': result_dept.get('exists', 0),
        'failed': result_dept.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Department And Desgination Mast'
    })

if 'result_desig' in locals():
    summary_data.append({
        'module': 'Designations',
        'created': result_desig.get('created', 0),
        'exists': result_desig.get('exists', 0),
        'failed': result_desig.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Department And Desgination Mast'
    })

if 'result_ct' in locals():
    summary_data.append({
        'module': 'Complaint Types',
        'created': result_ct.get('created', 0),
        'exists': result_ct.get('exists', 0),
        'failed': result_ct.get('failed', 0),
        'excel_file': COMMON_MASTER_FILE if 'COMMON_MASTER_FILE' in locals() else None,
        'sheet': 'Complaint Type Master'
    })

if 'result_employees' in locals():
    summary_data.append({
        'module': 'Employees (HRMS)',
        'created': result_employees.get('created', 0),
        'exists': result_employees.get('exists', 0),
        'failed': result_employees.get('failed', 0),
        'excel_file': EMPLOYEE_MASTER_FILE if 'EMPLOYEE_MASTER_FILE' in locals() else None,
        'sheet': 'Employee Master'
    })

# Calculate totals
total_created = sum(item['created'] for item in summary_data)
total_exists = sum(item['exists'] for item in summary_data)
total_failed = sum(item['failed'] for item in summary_data)
total_records = total_created + total_exists + total_failed

# Build HTML table
table_rows = []
for item in summary_data:
    total_for_module = item['created'] + item['exists'] + item['failed']
    table_rows.append(f"""
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">{item['module']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: green; font-weight: bold;">{item['created']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: orange; font-weight: bold;">{item['exists']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; color: red; font-weight: bold;">{item['failed']}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; font-weight: bold;">{total_for_module}</td>
        </tr>
    """)

timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Build download links section for updated Excel files
updated_files = {}
for item in summary_data:
    if item['excel_file'] and os.path.exists(item['excel_file']):
        file_path = item['excel_file']
        file_name = os.path.basename(file_path)
        if file_path not in updated_files:
            updated_files[file_path] = {
                'name': file_name,
                'sheets': [],
                'modules': [],
                'has_errors': False,
                'has_success': False
            }
        updated_files[file_path]['sheets'].append(item['sheet'])
        updated_files[file_path]['modules'].append(item['module'])
        if item['failed'] > 0:
            updated_files[file_path]['has_errors'] = True
        if item['created'] > 0 or item['exists'] > 0:
            updated_files[file_path]['has_success'] = True

# Generate download links HTML
download_links_html = ""

# Check for legacy error file (from old approach)
legacy_error_file = 'errors/FAILED_RECORDS.xlsx'
has_legacy_errors = os.path.exists(legacy_error_file)

if updated_files:
    download_links_html = """
    <div style="margin: 20px 0; padding: 15px; background-color: #e7f3ff; border-left: 4px solid #007bff; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #004085;">📥 Updated Excel Files with Status Columns</h3>
        <p style="color: #004085; margin-bottom: 15px;">
            The following Excel files have been updated with <b>_STATUS</b>, <b>_STATUS_CODE</b>, and <b>_ERROR_MESSAGE</b> columns:
        </p>
    """
    
    for file_path, file_info in updated_files.items():
        # Determine status badge
        if file_info['has_errors']:
            status_badge = '<span style="background-color: #dc3545; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">⚠️ HAS ERRORS</span>'
        else:
            status_badge = '<span style="background-color: #28a745; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px;">✅ ALL SUCCESS</span>'
        
        modules_list = ', '.join(file_info['modules'])
        
        download_links_html += f"""
        <div style="margin-bottom: 15px; padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
            <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap;">
                <div style="flex: 1; min-width: 300px;">
                    <strong style="color: #007bff;">📄 {file_info['name']}</strong> {status_badge}
                    <br>
                    <span style="font-size: 12px; color: #666;">Modules: {modules_list}</span>
                </div>
                <a href="{file_path}" download="{file_info['name']}" 
                   style="display: inline-block; padding: 8px 16px; background-color: #007bff; color: white; 
                          text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px; margin-top: 5px;">
                    ⬇️ Download Updated File
                </a>
            </div>
        </div>
        """
    
    download_links_html += """
        <p style="color: #004085; font-size: 12px; margin-top: 15px; border-top: 1px solid #bee5eb; padding-top: 10px;">
            <b>💡 How to use:</b>
        </p>
        <ul style="font-size: 12px; color: #004085; margin: 5px 0;">
            <li><span style="color: green; font-weight: bold;">🟢 GREEN rows (SUCCESS):</span> Successfully created in MDMS/HRMS</li>
            <li><span style="color: orange; font-weight: bold;">🟡 YELLOW rows (EXISTS):</span> Already exist in system (duplicates)</li>
            <li><span style="color: red; font-weight: bold;">🔴 RED rows (FAILED):</span> Failed to create - check _ERROR_MESSAGE column for details</li>
        </ul>
        <p style="font-size: 12px; color: #004085; margin-top: 10px;">
            <b>🔒 Note:</b> Status columns are <b>protected</b> and cannot be edited. Fix data in other columns and re-upload if needed.
        </p>
    </div>
    """

# Add legacy error file section if it exists
if has_legacy_errors and total_failed > 0:
    download_links_html += f"""
    <div style="margin: 20px 0; padding: 15px; background-color: #fff3cd; border-left: 4px solid #ffc107; border-radius: 5px;">
        <h3 style="margin-top: 0; color: #856404;">⚠️ Legacy Error File Detected</h3>
        <p style="color: #856404; margin-bottom: 10px;">
            An error file was generated from a previous upload approach. You can download it below:
        </p>
        <div style="padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
            <div style="display: flex; justify-content: space-between; align-items: center;">
                <div>
                    <strong style="color: #856404;">📄 FAILED_RECORDS.xlsx</strong>
                    <br>
                    <span style="font-size: 12px; color: #666;">Contains {total_failed} failed record(s)</span>
                </div>
                <a href="{legacy_error_file}" download="FAILED_RECORDS.xlsx" 
                   style="display: inline-block; padding: 8px 16px; background-color: #dc3545; color: white; 
                          text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px;">
                    ⬇️ Download Error File
                </a>
            </div>
        </div>
        <p style="color: #856404; font-size: 12px; margin-top: 10px;">
            <b>💡 Tip:</b> Use the updated Excel files above instead - they have status columns directly in your original templates!
        </p>
    </div>
    """

html_content = f"""
<div style="font-family: Arial, sans-serif; padding: 20px; border: 2px solid #007bff; border-radius: 10px; background-color: #f8f9fa;">
    <h2 style="color: #007bff; margin-top: 0;">📊 Data Upload Summary Report</h2>
    <p style="color: #666; margin-bottom: 20px;">Generated: {timestamp}</p>
    
    <table style="width: 100%; border-collapse: collapse; margin-bottom: 20px; background-color: white;">
        <thead>
            <tr style="background-color: #007bff; color: white;">
                <th style="padding: 12px; border: 1px solid #ddd; text-align: left;">Module</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">✅ Created</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">⚠️ Already Exists</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">❌ Failed</th>
                <th style="padding: 12px; border: 1px solid #ddd; text-align: center;">📊 Total</th>
            </tr>
        </thead>
        <tbody>
            {''.join(table_rows)}
            <tr style="background-color: #e9ecef; font-weight: bold;">
                <td style="padding: 12px; border: 1px solid #ddd;">TOTAL</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: green;">{total_created}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: orange;">{total_exists}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center; color: red;">{total_failed}</td>
                <td style="padding: 12px; border: 1px solid #ddd; text-align: center;">{total_records}</td>
            </tr>
        </tbody>
    </table>
    
    <div style="display: flex; justify-content: space-around; margin: 20px 0; flex-wrap: wrap;">
        <div style="text-align: center; padding: 15px; background-color: #d4edda; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #155724;">{total_created}</div>
            <div style="color: #155724;">Created</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #fff3cd; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #856404;">{total_exists}</div>
            <div style="color: #856404;">Already Exists</div>
        </div>
        <div style="text-align: center; padding: 15px; background-color: #f8d7da; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
            <div style="font-size: 32px; font-weight: bold; color: #721c24;">{total_failed}</div>
            <div style="color: #721c24;">Failed</div>
        </div>
    </div>
    
    {download_links_html}
</div>
"""

display(HTML(html_content))

print("\n🎉 DATA UPLOAD COMPLETED!")
if updated_files:
    print(f"\n📥 {len(updated_files)} Excel file(s) updated with status columns")
    for file_path in updated_files.keys():
        print(f"   • {os.path.basename(file_path)}")